In [ ]:
# DataFrame, Stream

In [1]:
import findspark
findspark.init()
findspark.find()

'/Users/chiholee/02.Project/spark/spark'

In [2]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""
os.environ['AWS_DEFAULT_REGION'] = ""

In [90]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Read S3 Parquet") \
    .getOrCreate()

In [5]:
spark.conf.set("spark.hadoop.fs.s3a.endpoint", "s3.ap-northeast-2.amazonaws.com")
spark.conf.set("spark.hadoop.fs.s3a.impl",
               "org.apache.hadoop.fs.s3a.S3AFileSystem")


In [24]:
df = spark.read \
    .format("parquet") \
    .option("header", True) \
    .load("s3a://seminar-s3/accesslog/2023/04/02/10/")
    # .load("s3a://seminar-s3/accesslog/2023/04/02/10/seminar-firehose-1-2023-04-02-10-24-55-9a49154d-e4f1-418f-9e4e-a154418b39c9.parquet")

    

In [26]:
dataSchema = df.schema
dataSchema

StructType([StructField('timestamp', TimestampType(), True), StructField('client_ip', StringType(), True), StructField('cust_id', LongType(), True), StructField('prd_id', LongType(), True), StructField('service', StringType(), True), StructField('http_referer', StringType(), True), StructField('dt', StringType(), True)])

In [41]:
streaming = spark.readStream\
                .schema(dataSchema)\
                .load("s3a://seminar-s3/accesslog/2023/04/02/10/")


In [67]:
spark.conf.set("spark.sql.adaptive.enabled",False)

In [98]:
streaming.createOrReplaceTempView("accesslog")

In [115]:
test_01 = spark.sql("""
select service customer_click, count(*) cnt
from accesslog
group by service
order by cnt
""").writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("sql_02") \
    .start()
    # .start().awaitTermination()

23/04/02 20:26:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/wg/dx6n4cld3kg_d3gcthzc01km0000gr/T/temporary-b05163ee-123f-4530-877e-4f94ce712e8a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [118]:
from IPython.display import display, clear_output
import time

while True:
    clear_output(wait=True)
    display(spark.sql('SELECT customer_click, cnt FROM sql_02').show())
    time.sleep(1)

+--------------+-----+
|customer_click|  cnt|
+--------------+-----+
|         order|  129|
|        review| 1370|
|        basket| 2266|
|         click|18145|
+--------------+-----+



None

KeyboardInterrupt: 